In [15]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [6]:
data = pd.read_csv('C:\\Users\\User\\PI_ML_OPS\\EDA\\data_merged.csv')
data_steam_juegos = data.copy()
data_random_forest = data.copy()

In [8]:
data_steam_juegos.drop(['Id','Id_user','Developer','Genres','Posted','Sentiment_analysis','Recommend','Price','Playtime_forever','Release_date'],axis=1,inplace=True)
data.drop(['Id','Id_user','Genres','Posted','Sentiment_analysis','Recommend','Price','Playtime_forever'],axis=1,inplace=True)



In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32514 entries, 0 to 32513
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id_item       32514 non-null  int64  
 1   App_name      32514 non-null  object 
 2   Developer     32514 non-null  object 
 3   Release_date  32514 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1016.2+ KB


In [10]:
data_steam_juegos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32514 entries, 0 to 32513
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id_item   32514 non-null  int64 
 1   App_name  32514 non-null  object
dtypes: int64(1), object(1)
memory usage: 508.2+ KB


In [11]:
data['Id_item'] = data['Id_item'].astype(str)
data['Release_date'] = data['Release_date'].astype(str)
# borrando duplicados
data = data.drop_duplicates(subset='Id_item', keep='first')
data_steam_juegos = data_steam_juegos.drop_duplicates(subset='Id_item', keep='first')
data['Especificaciones'] = data[['App_name','Developer','Release_date']].apply(lambda x: ', '.join(x), axis=1)
data.drop(['App_name','Developer','Release_date'],axis=1,inplace=True)

data.to_csv('C:\\Users\\User\\PI_ML_OPS\\Machine_Learning\\steam_juegos.csv', index=False)
data_steam_juegos.to_csv('C:\\Users\\User\\PI_ML_OPS\\Machine_Learning\\steam_id.csv', index=False)


Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recomendacion_juego(Id_item):
    try:
        # Cargar datos
        data = pd.read_csv('C:\\Users\\User\\PI_ML_OPS\\Machine_Learning\\steam_juegos.csv')
        data_steam_juegos = pd.read_csv('C:\\Users\\User\\PI_ML_OPS\\Machine_Learning\\steam_id.csv')

        # Verificar si el Id_item existe en el conjunto de datos
        if Id_item not in data['Id_item'].values:
            raise ValueError(f"El Id_item {Id_item} no existe en el conjunto de datos.")

        # Crear matriz TF-IDF usando la columna Especificaciones
        matriz = TfidfVectorizer(min_df=1, max_df=0.8, token_pattern=r'(?u)\b\w\w+\b')
        vector = matriz.fit_transform(data['Especificaciones'])
        df_vector = pd.DataFrame(vector.toarray(), index=data['Id_item'], columns=matriz.get_feature_names_out())

        # Calcular similitud de coseno
        vector_similitud_coseno = cosine_similarity(df_vector.values)
        cos_similarity_df = pd.DataFrame(vector_similitud_coseno, index=df_vector.index, columns=df_vector.index)

        # Verificar si el Id_item tiene similitud con otros juegos
        if Id_item not in cos_similarity_df.index:
            raise ValueError(f"No hay juegos similares al Id_item {Id_item}.")

        # Obtener recomendaciones y realizar operaciones
        recomendacion_juego = cos_similarity_df.loc[Id_item]
        recomendacion = recomendacion_juego.sort_values(ascending=False)
        resultado = recomendacion.head(6).reset_index()
        df_resultado = resultado.merge(data_steam_juegos, on='Id_item', how='left')

        # Verificar si el Id_item existe en el conjunto de datos de steam_id
        if Id_item not in data_steam_juegos['Id_item'].values:
            raise ValueError(f"El Id_item {Id_item} no existe en el conjunto de datos de steam_id.")

        # Obtener nombre del juego
        nombre_juego = data_steam_juegos[data_steam_juegos['Id_item'] == Id_item]['App_name'].values[0]

        # Crear mensaje de recomendación
        texto_recomendacion = f"Debido a que te gusta el juego {Id_item} : {nombre_juego}, estoy seguro que te gustarán los siguientes juegos "
                
        # Crear y devolver el resultado
        result = {
            'mensaje': texto_recomendacion.strip(),
            'Recomendaciones de juegos': df_resultado['App_name'][1:6].tolist()
        }

        return result

    except Exception as e:
        return {'error': str(e)}

# Ejemplo de uso
recomendaciones = recomendacion_juego(22000)  # Cambié el Id_item a string para coincidir con el formato
print(recomendaciones)


{'mensaje': 'Debido a que te gusta el juego 22000 : World of Goo, estoy seguro que te gustarán los siguientes juegos', 'Recomendaciones de juegos': ['Grey Goo', 'Stellar 2D', 'Call of Duty: World at War', 'S.T.A.L.K.E.R.: Clear Sky', 'Awesomenauts - the 2D moba']}


In [16]:
# Seleccionar características (X) y la etiqueta (y)
X = data_random_forest[['Id_item', 'Release_date', 'Price', 'Posted', 'Sentiment_analysis', 'Playtime_forever']]
y = data_random_forest['Recommend']

# Divide el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crea y entrena el modelo
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Realiza predicciones en el conjunto de prueba
predicciones = modelo.predict(X_test)

# Evalúa la precisión del modelo
precision = accuracy_score(y_test, predicciones)
print(f'Precisión del modelo: {precision}')



Precisión del modelo: 0.9061971397816393


In [38]:
def recomendacion_usuario(id_usuario):
    # Verifica si el usuario existe en el conjunto de datos
    if id_usuario not in data_random_forest['Id_user'].values:
        print(f'El usuario {id_usuario} no existe en el conjunto de datos.')
        return None

    # Filtra el conjunto de datos para obtener las características de juegos no etiquetados para el usuario
    juegos_sin_etiqueta = data_random_forest[data_random_forest['Id_user'] == id_usuario][['Id_item', 'Release_date', 'Price', 'Posted', 'Sentiment_analysis', 'Playtime_forever']]

    # Asegúrate de que haya al menos un juego sin etiquetar para el usuario
    if juegos_sin_etiqueta.empty:
        print(f'No hay juegos sin etiquetar para el usuario {id_usuario}.')
        return None

    # Utiliza el modelo entrenado para predecir las preferencias del usuario para los juegos sin etiquetar
    preferencias_usuario = modelo.predict(juegos_sin_etiqueta)

    # Combina las predicciones con la información del juego y selecciona los 5 mejores
    juegos_sin_etiqueta['Recommend'] = preferencias_usuario
    juegos_recomendados = juegos_sin_etiqueta.sort_values(by='Recommend', ascending=False).head(5)

    # Realiza una fusión con el conjunto de datos original para obtener el nombre del juego
    juegos_recomendados = pd.merge(juegos_recomendados, data_random_forest[['Id_item', 'App_name']], on='Id_item', how='left')

    # Elimina duplicados basados en 'App_name'
    juegos_recomendados = juegos_recomendados.drop_duplicates(subset='App_name')

    # Reinicia el índice y luego incrementa en 1
    juegos_recomendados.reset_index(drop=True, inplace=True)
    juegos_recomendados.index += 1

    return juegos_recomendados[['App_name','Release_date','Price','Sentiment_analysis','Playtime_forever']]

# Llama a la función con el ID de usuario para obtener las recomendaciones
id_usuario = 'DemonPieOfTheNight'  # Reemplaza con el ID real de usuario que quieras obtener recomendaciones
recomendaciones = recomendacion_usuario(id_usuario)

# Imprime las recomendaciones
if recomendaciones is not None:
    print(f'Recomendaciones para el usuario {id_usuario}:\n{recomendaciones}')




Recomendaciones para el usuario DemonPieOfTheNight:
                      App_name  Release_date  Price  Sentiment_analysis  \
1                     Altitude        2009.0   0.00                   1   
2       Swords and Soldiers HD        2010.0   9.99                   2   
3                       Portal        2007.0   9.99                   1   
4  Sid Meier's Civilization® V        2010.0  29.99                   2   
5             Ms. Splosion Man        2013.0   9.99                   2   

   Playtime_forever  
1               931  
2               345  
3                95  
4              1862  
5               334  
